In [73]:
import spacy
from spacy import displacy
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

In [74]:
path_JuanSe = "/Users/juan/Documents/Juan's MacBook Pro/CUNEF/Quinto/Informacion no Estructurada/Practica Final/Data_pacifico_vallecas_info_no_estruct.xlsx"

df = pd.read_excel(path_JuanSe)

In [75]:
df.head()

,title,price,info_features,comment,details
0,Piso en venta en Palomeras Bajas\nPuente de Va...,61.000 €,86 m²\n3 hab.\nPlanta -1 exterior sin ascensor,"VENTA DE LA NUDA PROPIEDAD, SOLO PARA INVERSOR...","Características básicas\n86 m² construidos, 81..."
1,Piso en venta en calle de Josué Lillo s/n\nSan...,112.000 €,86 m²\n3 hab.\nPlanta 1ª sin ascensor,Piso ocupado por persona sin justo título. Est...,"Características básicas\n86 m² construidos, 75..."
2,"Piso en venta en avenida Palomeras, 9\nPalomer...",112.100 €,110 m²\n2 hab.\nPlanta 1ª exterior sin ascensor,"Edificio situado en la ciudad de Madrid, provi...",Características básicas\n110 m² construidos\n2...
3,Piso en venta en calle de Peña Redonda\nPalome...,119.000 €,91 m²\n3 hab.\nBajo exterior sin ascensor,SERVICHECK INMOBILIARIA VENDE PISO DE 3 HABITA...,"Características básicas\n91 m² construidos, 80..."
4,Piso en venta en avenida de San Diego\nSan Die...,120.000 €,87 m²\n3 hab.\nPlanta 1ª exterior sin ascensor,"Vivienda situada en 1ªplanta, contamos con 87m...","Características básicas\n87 m² construidos, 71..."


In [76]:
def symbol_remover(col, symbollist, num = True):
    """
    Esta funcion sirve para formatear columnas numericas que incluyen simbolos
    
    Input: 
    col : columna a formatear
    symbollist : lista de simbolos a eliminar
    num : si la variable final se quiere convertir a numerica
    
    Output: columna con valores libre de simbolos y convertida en numerica si procede
    """
    
    for s in symbollist:
        col = col.str.replace(s, '')
    if num == True: 
        col = col.astype(float)
    
    return col

In [77]:
df["info_features"] = df["info_features"].str.split('\n')
df["details"] = df["details"].str.split('\n')


In [78]:
df["price"] = symbol_remover(df["price"], [".","€"])

In [79]:
for i in range(len(df)):
    if not len(df["info_features"][i]) == 0:
            if df["info_features"][i][0].startswith(("Obra","Urge")):
                del df["info_features"][i][0]

In [82]:
def feature_create(identifier, criteria = "start", value = "bool"):
    """
    Esta funcion crea una columna con la informacion deseada de la columna info_features
    
    Imput:
    identifier: en que string se basa para encontrar el dato
    criteria: si el string lo busca al principio o al final del elemento
    value: "bool"/"value" si se desea que el la funcion devuelva el valor encontrado 
            o un booleano si se encuentra el valor buscado
            Por ejemplo en m2 busca el valor, mientras que para ascensor Bool de si hay en el piso.
    
    """
    blank = []

    for i in range(len(df)):
        if not len(df["info_features"][i]) == 0:
                if (criteria == "start" and df["info_features"][i][0].startswith(identifier)) or \
                     (criteria == "end" and df["info_features"][i][0].endswith(identifier)):
                    if value == "bool":
                        blank.append(True)
                    if value == "value":
                        blank.append(df["info_features"][i][0])
                    del df["info_features"][i][0]
                else: blank.append(False)
        else: blank.append(False)
    

    return blank

In [83]:
df["m2"] = feature_create(identifier = "m²", criteria = "end", value = "value")
df["m2"] = symbol_remover(df["m2"], [" m²"])

In [65]:
df["hab"] = feature_create(identifier = "hab.", criteria = "end", value = "value")
df["hab"] = symbol_remover(df["hab"], [" hab."])

In [69]:
df["info_features"]

0                      [Planta -1 exterior sin ascensor]
1                               [Planta 1ª sin ascensor]
2                      [Planta 1ª exterior sin ascensor]
3                           [Bajo exterior sin ascensor]
4                      [Planta 1ª exterior sin ascensor]
                             ...                        
230    [Entreplanta exterior con ascensor, Garaje inc...
231                    [Planta 9ª exterior con ascensor]
232                              [exterior con ascensor]
233                              [exterior con ascensor]
234                    [Planta 4ª exterior con ascensor]
Name: info_features, Length: 235, dtype: object

In [9]:
for i in range(len(df)):
    df["info_features"][i][0] = df["info_features"][i][0].replace("Semi-sótano ", "Planta -1 ")
    df["info_features"][i][0] = df["info_features"][i][0].replace("Bajo ", "Planta 0 ")
    df["info_features"][i][0] = df["info_features"][i][0].replace("Entreplanta ", "Planta 0 ")
    if not df["info_features"][i][0].startswith("Planta "):
        df["info_features"][i][0] = df["info_features"][i][0].replace("exterior ", "Planta NA exterior ")

In [10]:
df["planta"] = df["info_features"].str[0]
df["planta"] = symbol_remover(df["planta"], ["Planta ", " ascensor", "ª"], num=False)
df["info_features"]=df["info_features"].str[1:]

In [11]:
df["exterior"] = df["planta"].str.contains('exterior')
df["ascensor"] = df["planta"].str.contains('con')
df["planta"] = df["planta"].str[:2]

In [12]:
blank = []
for i in range(len(df)):
    
    if len(df["info_features"][i]) == 0:
        blank.append(False)
            
    else: 
        if df["info_features"][i][0].startswith("G"):
                blank.append(True)
                del df["info_features"][i][0]
        else: blank.append(False)

df["garaje"] = blank


In [13]:
blank = []
for i in range(len(df)):
    if not len(df["info_features"][i]) == 0:
            if df["info_features"][i][0].startswith("Lu"):
                blank.append(True)
                del df["info_features"][i][0]
            else: blank.append(False)
    else: blank.append(False)

df["lujo"] = blank


In [14]:
blank = []
for i in range(len(df)):
    if not len(df["info_features"][i]) == 0:
            if df["info_features"][i][0].startswith("Lo"):
                blank.append(True)
                del df["info_features"][i][0]
            else: blank.append(False)
    else: blank.append(False)

df["loft"] = blank


In [15]:
del df["info_features"]

In [70]:
df.head()

,title,price,info_features,comment,details,m2,hab
0,Piso en venta en Palomeras Bajas\nPuente de Va...,61000.0,[Planta -1 exterior sin ascensor],"VENTA DE LA NUDA PROPIEDAD, SOLO PARA INVERSOR...","[Características básicas, 86 m² construidos, 8...",86.0,3.0
1,Piso en venta en calle de Josué Lillo s/n\nSan...,112000.0,[Planta 1ª sin ascensor],Piso ocupado por persona sin justo título. Est...,"[Características básicas, 86 m² construidos, 7...",86.0,3.0
2,"Piso en venta en avenida Palomeras, 9\nPalomer...",112100.0,[Planta 1ª exterior sin ascensor],"Edificio situado en la ciudad de Madrid, provi...","[Características básicas, 110 m² construidos, ...",110.0,2.0
3,Piso en venta en calle de Peña Redonda\nPalome...,119000.0,[Bajo exterior sin ascensor],SERVICHECK INMOBILIARIA VENDE PISO DE 3 HABITA...,"[Características básicas, 91 m² construidos, 8...",91.0,3.0
4,Piso en venta en avenida de San Diego\nSan Die...,120000.0,[Planta 1ª exterior sin ascensor],"Vivienda situada en 1ªplanta, contamos con 87m...","[Características básicas, 87 m² construidos, 7...",87.0,3.0


In [71]:
df.to_excel("/Users/juan/Documents/Juan's MacBook Pro/CUNEF/Quinto/Informacion no Estructurada/Practica Final/Data_pacifico_vallecas_info_no_estruct_PROC.xlsx")